# 타이타닉 생존 분석: 데이터 전처리

## 1. 개요
이 노트북에서는 타이타닉 생존 분석을 위한 데이터 전처리를 수행합니다.

### 1.1 전처리 목표
- 결측치 처리
- 불필요한 컬럼 제거
- 파생변수 생성
- 데이터 타입 최적화

### 1.2 필요한 라이브러리

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
plt.style.use('classic')
sns.set_theme()
%matplotlib inline

### 1.3 데이터 로드

In [2]:
import sys
sys.path.append('..')
from src.data.load_data import download_titanic_data

df = download_titanic_data()

Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\data/raw
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\data/processed
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\data/external
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\notebooks
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\src/data
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\src/features
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\src/visualization
Directory checked/created: c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\reports/figures
Data successfully downloaded and saved to c:\Users\kjhgr\OneDrive - 인하공업전문대학\바탕 화면\titanic_analysis\data/raw/titanic.csv
Shape of the dataset: (891, 15)

Basic information about the dataset:
<class 'pandas.core.frame

## 2. 데이터 탐색 및 이해

### 2.1 데이터셋 기본 정보
- 총 891개의 데이터 포인트
- 15개의 특성(컬럼)
- 다양한 데이터 타입 (bool, category, float64, int64, object)

### 2.2 컬럼별 특성
1. 수치형 변수:
   - survived (int64): 생존 여부 (0/1)
   - pclass (int64): 객실 등급 (1/2/3)
   - age (float64): 나이
   - sibsp (int64): 형제/배우자 수
   - parch (int64): 부모/자녀 수
   - fare (float64): 요금

2. 범주형 변수:
   - sex (object): 성별
   - embarked (object): 승선 항구
   - class (category): 객실 등급 (문자형)
   - who (object): 승객 구분
   - deck (category): 선실 갑판
   - embark_town (object): 승선 도시
   - alive (object): 생존 여부 (문자형)

3. 불리언 변수:
   - adult_male (bool): 성인 남성 여부
   - alone (bool): 혼자 탑승 여부

### 2.3 결측치 현황
- age: 177개 결측 (19.9%)
- embarked: 2개 결측 (0.2%)
- deck: 688개 결측 (77.2%)
- embark_town: 2개 결측 (0.2%)

### 2.4 데이터 전처리 계획
1. 중복 정보 처리:
   - class와 pclass: pclass만 유지
   - alive와 survived: survived만 유지
   - embark_town과 embarked: embarked만 유지

2. 결측치 처리 전략:
   - age: 평균값으로 대체
   - embarked: 최빈값으로 대체
   - deck: 'Unknown' 범주 생성

3. 파생변수 생성 계획:
   - family_size: sibsp + parch + 1
   - family_type: 가족 규모 기준 범주화
   - age_group: 연령대별 그룹화
   - fare_category: 요금 구간 범주화

## 3. 데이터 전처리 실행

### 3.1 데이터 전처리 함수 정의
전처리 작업을 체계적으로 수행하기 위한 함수를 정의합니다.


In [29]:
def preprocess_titanic(df):
    """
    타이타닉 데이터셋 전처리 함수
    
    Parameters:
    df (pandas.DataFrame): 원본 타이타닉 데이터셋
    
    Returns:
    pandas.DataFrame: 전처리된 데이터셋
    """
    # 데이터 복사
    df_processed = df.copy()
    
    # 1. 불필요한 중복 컬럼 제거
    columns_to_drop = ['class', 'alive', 'embark_town']
    df_processed = df_processed.drop(columns=columns_to_drop)
    
    # 2. 결측치 처리
    df_processed['age'] = df_processed['age'].fillna(df_processed['age'].mean())
    df_processed['embarked'] = df_processed['embarked'].fillna(df_processed['embarked'].mode()[0])
    df_processed['deck'] = df_processed['deck'].astype(str).replace('nan', 'Unknown')
    
    # 3. 파생변수 생성
    # 가족 크기
    df_processed['family_size'] = df_processed['sibsp'] + df_processed['parch'] + 1
    
    # 가족 유형
    df_processed['family_type'] = pd.cut(df_processed['family_size'], 
                                       bins=[0, 1, 4, float('inf')], 
                                       labels=['Alone', 'Small Family', 'Large Family'])
    
    # 나이 그룹
    df_processed['age_group'] = pd.cut(df_processed['age'], 
                                     bins=[0, 12, 20, 40, 60, float('inf')],
                                     labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])
    
    # 요금 카테고리
    df_processed['fare_category'] = pd.qcut(df_processed['fare'], 
                                          q=4, 
                                          labels=['Low', 'Medium', 'High', 'Very High'])
    
    # 4. 데이터 타입 최적화
    # 범주형 변수 변환
    categorical_columns = ['sex', 'embarked', 'deck', 'who', 'family_type', 
                         'age_group', 'fare_category']
    for col in categorical_columns:
        df_processed[col] = df_processed[col].astype('category')
    
    return df_processed

# 전처리 실행
df_processed = preprocess_titanic(df)

# 전처리된 데이터 확인
print("전처리된 데이터 정보:")
print(df_processed.info())

# 처리된 데이터 샘플 확인
print("\n처리된 데이터 샘플:")
print(df_processed.head())

# 결측치 재확인
print("\n결측치 확인:")
print(df_processed.isnull().sum())

전처리된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   survived       891 non-null    int64   
 1   pclass         891 non-null    int64   
 2   sex            891 non-null    category
 3   age            891 non-null    float64 
 4   sibsp          891 non-null    int64   
 5   parch          891 non-null    int64   
 6   fare           891 non-null    float64 
 7   embarked       891 non-null    category
 8   who            891 non-null    category
 9   adult_male     891 non-null    bool    
 10  deck           891 non-null    category
 11  alone          891 non-null    bool    
 12  family_size    891 non-null    int64   
 13  family_type    891 non-null    category
 14  age_group      891 non-null    category
 15  fare_category  891 non-null    category
dtypes: bool(2), category(7), float64(2), int64(5)
memory usage: 58.0 KB

## 4. 전처리 결과 검증 및 요약

### 4.1 전처리 완료 사항
1. 데이터셋 구조
   - 891개 행, 16개 컬럼 유지
   - 모든 결측치 처리 완료
   - 데이터 타입 최적화 (bool, category, float64, int64)

2. 변수 변환 결과
- 제거된 컬럼: 'class', 'alive', 'embark_town'
- 범주형으로 변환된 변수들:
  * sex
  * embarked
  * who
  * deck
  * family_type
  * age_group
  * fare_category

3. 새로 생성된 변수
- family_size: 전체 가족 구성원 수
- family_type: 가족 규모별 분류
- age_group: 연령대별 분류
- fare_category: 요금 구간 분류

### 4.2 메모리 사용 최적화
- 메모리 사용량: 58.0 KB
- 범주형 변수 변환으로 메모리 효율성 개선

### 4.3 다음 단계 준비
전처리된 데이터를 저장하고 분석 단계로 넘어갑니다.

In [ ]:
# 전처리된 데이터 저장
output_path = 'data/processed/titanic_processed.csv'
df_processed.to_csv(output_path, index=False)
print(f"전처리된 데이터가 {output_path}에 저장되었습니다.")

NameError: name '__file__' is not defined